# About

This is a notebook for finding and copying Textures form extracted game assets to named images for the item database. 

## Why not a script?

because depending on what extractor you use and the whims of the developers all this could use some serious tweaking every run. The notebook lets you run things in stages and inspect what your working with.

In [1]:
import json

database = {}

with open("data/database.json", "r") as f:
    database = json.load(f)

db = database["db"]

Item Database Pulled in

In [2]:
from pathlib import Path 

# Location were https://github.com/SeriousCache/UABE has extracted all Texture2D assets
# Change as necessary
datapath = Path(r"E:\Games\SteamLibrary\steamapps\common\Stationeers\Stationpedia\exported_textures")


Change this Datapath to point to the extracted textures

In [3]:
import os

# Pull in a list of all found textures
images = list(datapath.glob("*.png"))
names = [image.name  for image in images]


### Finding matches

This next section loops through all the item names and collects all the candidate textures. Then, through a series of rules, attempts to narrow down the choices to 1 texture.

In [4]:
image_candidates = {}

def filter_candidates(candidates):
    max_match_len = 0
    filtered_matches = []

    # go for longest match
    for can in candidates:
        name, match, mapping = can
        match_len = len(match)
        if match_len > max_match_len:
            max_match_len = match_len
            filtered_matches = [(name, match, mapping)]
        elif match_len == max_match_len:
            filtered_matches.append((name, match, mapping))

    # choose better matches
    if len(filtered_matches) > 1:
        better_matches = []
        for can in filtered_matches:
            name, match, mapping = can
            if mapping.startswith("Item") and mapping in name:
                better_matches.append((name, match, mapping))
            elif mapping.startswith("Structure") and mapping in name:
                better_matches.append((name, match, mapping))
        if len(better_matches) > 0:
            filtered_matches = better_matches

    #exclude build states if we have non build states
    if len(filtered_matches) > 1:
        non_build_state = []
        for can in filtered_matches:
            name, match, mapping = can
            if "BuildState" not in name:
                non_build_state.append((name, match, mapping))
        if len(non_build_state) > 0:
            filtered_matches = non_build_state

    #prefer matches without extra tags
    if len(filtered_matches) > 1:
        direct = []
        for can in filtered_matches:
            name, match, mapping = can
            if f"{match}-" in name:
                direct.append((name, match, mapping))
        if len(direct) > 0:
            filtered_matches = direct
    
    #filter to unique filenames
    if len(filtered_matches) > 1:
        unique_names = []
        unique_matches = []
        for can in filtered_matches:
            name, match, mapping = can
            if name not in unique_names:
                unique_names.append(name)
                unique_matches.append((name, match, mapping))
        filtered_matches = unique_matches

    #prefer not worse matches
    if len(filtered_matches) > 1:
        not_worse = []
        for can in filtered_matches:
            name, match, mapping = can
            if name.startswith("Item") and not mapping.startswith("Item"):
                continue
            elif name.startswith("Structure") and not mapping.startswith("Structure"):
                continue
            elif name.startswith("Kit") and not mapping.startswith("Kit"):
                continue
            elif not name.startswith(match):
                continue
            not_worse.append((name, match, mapping))
        if len(not_worse) > 0:
            filtered_matches = not_worse

    #if we have colored variants take White
    if len(filtered_matches) > 1:
        for can in filtered_matches:
            name, match, mapping = can
            if f"_White" in name:
                return [name]

    return [name for name, _, _ in filtered_matches]

for entry in db.values():
    candidates = []
    for name in names:
        if entry["name"] in name:
            candidates.append((name, entry["name"], entry["name"]))
        if entry["name"].removeprefix("Item") in name:
            candidates.append((name, entry["name"].removeprefix("Item"), entry["name"]))
        if entry["name"].removeprefix("Structure") in name:
            candidates.append((name, entry["name"].removeprefix("Structure"), entry["name"]))
    image_candidates[entry["name"]] = filter_candidates(candidates)



Some Items end up with no match but these items are often subtypes of an item that will have a match

In [5]:
# rematch items to super structure?
for name in image_candidates.keys():
    for other in image_candidates.keys():
        if name != other and name in other:
            if len(image_candidates[name]) > 0 and len(image_candidates[other]) == 0:
                image_candidates[other] = image_candidates[name]

Prepare out List of file copies. at this point a few items will never have a match. and one or two will have two choices but those choices will be arbitrary.

In [6]:
to_copy = []
for name, candidates in image_candidates.items():
    if len(candidates) != 1:
        print(name, candidates)
        if len(candidates) > 1:
            #take first as fallback
            to_copy.append((name, candidates[0]))
    else:
        # print(name, candidates)
        to_copy.append((name, candidates[0]))


ItemBiomass []
StructureBlocker []
CartridgePlantAnalyser []
StructureElevatorLevelIndustrial []
ItemPlantEndothermic_Creative []
Flag_ODA_10m []
Flag_ODA_4m []
Flag_ODA_6m []
Flag_ODA_8m []
ItemHorticultureBelt []
ItemKitLiquidRegulator []
ItemKitPortablesConnector []
Landingpad_GasConnectorInwardPiece []
Landingpad_LiquidConnectorInwardPiece []
ItemMushroom ['ItemMushroom-resources.assets-3022.png', 'ItemMushroom-resources.assets-9304.png']
StructurePlinth []
ItemPlantThermogenic_Creative []


In [11]:
import shutil

destpath = Path("img/stationpedia")
total_files = len(to_copy)

count = 0
print ( f"{count} of {total_files} | { count / total_files * 100}", end="\r")
for name, file in to_copy:
    source = datapath / file
    dest = destpath / f"{name}.png"
    shutil.copy(source, dest)
    count += 1
    print ( f"{count} of {total_files} | { (count / total_files) * 100 :.2f}%    ",  end="\r")
print()
print("Done")



1223 of 1223 | 100.00%    
Done
